Q1.1  Web2String 服务 
安装requests、flask框架

In [1]:
pip install requests

In [2]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests as r
from flask import Flask, request

app = Flask(__name__)

def Web2string(url:str) -> str:
    req = r.request('GET',url)
    return req.text

@app.route('/Web2string/', methods = ['GET','POST'])
def welcome():
    if request.method == 'POST':
        url = requests.args.get('url')
        return Web2string(url)


if __name__ == '__main__':
    app.debug = True
    app.run(host = '0.0.0.0',port=2345)
    
def download_web_content(url:str) -> str:
    req = r.request('GET',url)
    return req.text


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\123\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
pip install wordfilter

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install profanity-filter

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests as r
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from typing import List, Any

In [8]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Patrick
[nltk_data]     YIP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Patrick
[nltk_data]     YIP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def web2string(url: str) -> str:
    reg = re.compile('<[^>]*>')
    req = r.get(url).content
    url = 'https://www.public.asu.edu/~ychen10/teaching/honors.html'
    content = reg.sub('', req.decode("utf-8")).replace('\n', '').replace('\r', ' ')
    return content


Q1.2  分析并筛选出功能词（停用词）
首先定义停用词并且得出网页的内容，再进行分析和筛选停用词

In [10]:
def word_filter(words: str) -> list[Any]:
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(words)
    filtered_sentence = ''
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence += ' ' + w
    
    return filtered_sentence

In [11]:
words_strings = web2string(
    'https://www.public.asu.edu/~ychen10/teaching/honors.html')
words_strings

"      Honors Resource Page for CSE492/493     Dr. Yinong Chen, Arizona State University     This pages shows a few resources that can help honors students to start defining their thesis topics and find faculty adviser who are in the thesis topic areas.                                    ASU Barrett Honors College                                                 Barrett Honors Faculty Advisers at Computer Science, Computer Systems Engineering, and Informatics                                                    CSE492 and CSE493 Honors Thesis Advising Page at SCAI.              A list of topics and SCAI faculty who offer Honor's theses                                                 Honors projects from other ASU schools                                                    Barret Faculty Honors Adviser Video Series: How do you choose your thesis project?                                                    All CSE Faculty Pages and Their Research Areas                                         

In [12]:
word_filter(words_strings)

" Honors Resource Page CSE492/493 Dr. Yinong Chen , Arizona State University This pages shows resources help honors students start defining thesis topics find faculty adviser thesis topic areas . ASU Barrett Honors College Barrett Honors Faculty Advisers Computer Science , Computer Systems Engineering , Informatics CSE492 CSE493 Honors Thesis Advising Page SCAI . A list topics SCAI faculty offer Honor 's theses Honors projects ASU schools Barret Faculty Honors Adviser Video Series : How choose thesis project ? All CSE Faculty Pages Their Research Areas Sample CSE493 Honors Thesis Projects chaired Dr. Yinong Chen Combine Honors Thesis project CSE Capstone Project Return Dr. Chen 's Home Page"

Q1.3  Top10Words
运用nltk的框架，分析前10个高频词汇

In [13]:
import nltk
allWords = nltk.tokenize.word_tokenize(words_strings)
allWordDist = nltk.FreqDist(w.lower() for w in allWords)

stopwords = nltk.corpus.stopwords.words('english')
allWordExceptStopDist = nltk.FreqDist(w.lower() for w in allWords if w not in stopwords)    

In [14]:
mostCommon= allWordDist.most_common(10)

In [15]:
print(mostCommon)

[('honors', 9), ('thesis', 6), ('and', 5), ('faculty', 5), ('page', 3), ('dr.', 3), ('chen', 3), (',', 3), ('your', 3), ('project', 3)]


Q2.1  在此问题中，你将在 ASP .Net 框架中实现 Web 应用程序或在 Xamarin 框架中实现手机应 用程序。应用程序的架构如图 1 所示。你将实现图 1 中的所有部件。因为在问题 1 中已经 实现了前一半的部件，在问题 2 主要是实现后一半的问题，并调用在问题 1 中开发的的服务来构建应用程序。

In [21]:
from flask import Flask, render_template, request, redirect, flash, url_for
from Web2String import get_web_content, remove_tags, words_filter, top_ten_words, find_urls, tf_idf

app = Flask(__name__)
app.secret_key = 'amazing'

@app.route('/')
@app.route('/form')
def form():
    return render_template('form.html')


@app.route('/answer', methods=['GET', 'POST'])
def answer():
    inputs = ''
    for key, value in request.form.items():
        if key == 'Input':
            inputs = value

    if not inputs:
        flash("Input is required!")
        return redirect(url_for('form'))

    content = get_web_content(inputs)

    text = remove_tags(inputs)

    filters = words_filter(text)

    topten_doc1 = top_ten_words(filters, 1, 10)

    topten_doc2, topten_doc3 = {}, {}

    links = find_urls(inputs)
    for i in range(len(links)):
        contents = remove_tags(links[i])
        filters = words_filter(contents)
        my_dict = top_ten_words(filters, i+1, 10)
        if i == 0:
            topten_doc2 = my_dict
        if i == 1:
            topten_doc3 = my_dict

    # TF-IDF
    top_ten_words_tf_idf = tf_idf()

    return render_template('form.html', content=content, filter=filters,
                           topten_doc1=topten_doc1, topten_doc2=topten_doc2, topten_doc3=topten_doc3,
                           top_ten_words_tf_idf=top_ten_words_tf_idf)


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=2345, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

下载网页内容，并且移除网页标签，过滤停用词并且展示前十个高频词。分别保存3个文档，计算出TF-IDF值(idf = math.log2(3/value))。

In [28]:
@app.route('/tf-idf')
def tf_idf() -> dict[str, float]:
    new_dict = {}
    res_dict = {}

for idx in range(3):
    file_name =  f'words_{abs(idx+1)}.txt'
    f = open(file=file_name,mode='r',encoding='utf-8')
    data = json.load(f)
    
    for key,value in data.items():
        if new_dict.get(key):
            new_dict[key] += 1
        else:
            new_dict[key] = 1

    for key,value in new_data.items():
        idf = math.log2(3/value)
        print('idf value:',idf)
        tf_idf = idf *value
        res_dict[key] = tf_idf

    res = sorted(res_dict.tems(),key=lambda kv:(kv[1],kv[0]), reverse = True)
    
    my_dict = dict((x,y) for x,y in res[:10])
    
my_dict

FileNotFoundError: [Errno 2] No such file or directory: 'words_1.txt'